# Import

In [16]:
import numpy as np
import json

# Classes

In [17]:
# Convolution
class Convolution:
    def __init__(self, kernel, padding, stride, nb_filter, fct_activation):
        self.kernel = kernel
        self.padding = padding
        self.stride = stride
        self.nb_filter = nb_filter
        self.fct_activation = fct_activation

In [18]:
# input
class InputLayer():
    def __init__(self, shape):
        self.shape = shape

In [19]:
# Pooling Avg/Max
class Pooling:
    def __init__(self, op, kernel=2, padding="valid", stride=None):
        self.op = op # is the operation wanted (avg/max)
        self.kernel = kernel
        self.padding = padding
        self.stride = stride

In [20]:
# Class Flatten
class Flatten:
    pass

In [21]:
# Dense --> Fully connected layer
class Dense:
    def __init__(self, nb_neurones, fct_activation):
        self.nb_neurones = nb_neurones
        self.fct_activation = fct_activation

# Functions

In [37]:
# function that create CNN with list of layer
def create_CNN(list_layers):
    str_model_cnn = "model = keras.models.Sequential([\n"
    # iterate List of layer
    for layer in list_layers:
        
        # if is Intput
        if isinstance(layer, InputLayer):
            str_model_cnn += "\t\tkeras.layers.Input({}),\n".format(
                    layer.shape)
            
            
        # if is Convolution
        elif isinstance(layer, Convolution):
            str_model_cnn += "\t\tkeras.layers.Conv2D({}, kernel_size={}, strides={}, activation='{}', padding={}),\n".format(
                    layer.nb_filter,
                    layer.kernel,
                    layer.stride,
                    layer.fct_activation,
                    layer.padding)
            
        # if is Pooling
        elif isinstance(layer, Pooling):
            if(layer.op == "avg"): # avg Pooling 
                str_model_cnn += "\t\tkeras.layers.AveragePooling2D(pool_size={}, strides={}, padding={}),\n".format(
                    layer.kernel,
                    layer.stride,
                    layer.padding)
                
            else : # Max Pooling
                str_model_cnn += "\t\tkeras.layers.MaxPooling2D(pool_size={}, strides={}, padding={}),\n".format(
                    layer.kernel,
                    layer.stride,
                    layer.padding)
                
        # if is Dense (aka Fully connected layer)
        elif isinstance(layer, Dense):
            str_model_cnn += "\t\tkeras.layers.Dense({}, activation='{}'),\n".format(
                layer.nb_neurones, 
                layer.fct_activation)
            
        # if is flatten
        elif isinstance(layer, Flatten):
            str_model_cnn += "\t\tkeras.layers.Flatten(),\n"
            
        # Not possible
        else : print("Not Possible")

    # end model 
    str_model_cnn += "\n\t])\n"
    return str_model_cnn

In [44]:
# read json file 
# return list of layer
def getArchitectureFromJsonFile(json_file):
    
    # get architecture by json file
    with open(json_file) as archi_json:
        archi = json.load(archi_json)

    # instantiate class
    list_layer = []
    for layer in archi:
        parameters = layer["parameters"] # get parameters
        # if is Input class
        if(layer["class"] == "InputLayer"):
            # instantiate class and add into the list
            list_layer.append(InputLayer(parameters["shape"]))

        
        # if is Pooling class
        elif(layer["class"] == "Pooling"):
            try:
                padding = int(parameters["padding"])
            except:
                padding = "'" + parameters["padding"] + "'"
                
            list_layer.append(Pooling(parameters["op"],
                                      parameters["kernel"],
                                      padding,
                                      parameters["stride"]))
            
        # if is Convolution class
        elif(layer["class"] == "Convolution"):
            try:
                padding = int(parameters["padding"])
            except:
                padding = "'" + parameters["padding"] + "'"
                
                
            list_layer.append(Convolution(parameters["kernel"],
                                          padding,
                                          parameters["stride"],
                                          parameters["nb_filter"],
                                          parameters["fct_activation"]))
            
        # if is Flatten class
        elif(layer["class"] == "Flatten"):
            list_layer.append(Flatten())
        
        # if is Dense class
        elif(layer["class"] == "Dense"):
            list_layer.append(Dense(parameters["nb_neurones"],
                                    parameters["fct_activation"]))

        else : print("Error")
    return list_layer

In [67]:
def create_py_file(json_file):
    s = json_file.split(".")
    file_name = s[0].split("/")[1]
    architecture = getArchitectureFromJsonFile(json_file)
    
    # python directory
    py_dir = "architecture_py/"
    
    # log directory
    log_dir = "../architecture_log/"
    
    # png directory
    png_dir = "../architecture_img/"
    
    # reset file
    file_py = open(py_dir + file_name + ".py", "w")
    file_py.close()
    
    file_py = open(py_dir + file_name + ".py", "a") # Open file in writting (a --> append)
    
    # write import
    file_py.write("""
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.utils import plot_model
import sys
import traceback
import csv
from time import time
""")
    
    # write train/test data 
    file_py.write("""(train_x, train_y), (test_x, test_y) = keras.datasets.mnist.load_data()

# normaliser les pixel 0-255 -> 0-1
train_x = train_x / 255.0
test_x = test_x / 255.0

train_x = tf.expand_dims(train_x, 3)
test_x = tf.expand_dims(test_x, 3)

val_x = train_x[:5000]
val_y = train_y[:5000]


# init training time
training_time = 0
# init result
result_loss = ""
result_acc = ""
""")
    
    # try
    file_py.write("""try:
    """)
    
    # write architecture model
    file_py.write(create_CNN(architecture))
    
    # write : create png of the model
    file_py.write("    plot_model(model, show_shapes=True, to_file=\"%s\")\n" % (png_dir+file_name+".png"))
    
    # write compiler
    file_py.write("""    model.compile(optimizer='adam', loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])\n""")
    
    # write way for time computation
    file_py.write("""    start = time()\n""")
    
    # write model training
    file_py.write("""    model.fit(train_x, train_y, epochs=5, validation_data=(val_x, val_y))\n""")
    
    # write time computation
    file_py.write("""    training_time = time()-start\n""")
    
    # write model evaluation
    file_py.write("""    print(model.evaluate(test_x, test_y))\n""")
    
    # all is great
    log_file = log_dir + file_name +".log"
    file_py.write("""
    print('OK: file """ + log_file +""" has been create')
    log_file = open(\"""" + log_file + """\" , "w")
    log_file.write(str(model.evaluate(test_x, test_y)))
    result_loss = model.evaluate(test_x, test_y)[0]
    result_acc = model.evaluate(test_x, test_y)[1]
    log_file.close()
""")
    
    # something go wrong 
    error_file = log_dir + file_name + "_error.log"
    file_py.write("""except:
    print('error: file """ + error_file +""" has been create')
    error_file = open(\"""" + error_file + """\" , "w")
    traceback.print_exc(file=error_file)
    result_loss = "Error"
    result_acc = "Error"
    error_file.close()
""")
    
    file_py.write("""finally:
    file = open('../architecture_results.csv', 'a', newline ='')
    with file: 

        # identifying header   
        header = ['file_name', 'training_time(s)', 'result_loss', 'result_acc'] 
        writer = csv.DictWriter(file, fieldnames = header) 
      
        # writing data row-wise into the csv file 
        # writer.writeheader() 
        writer.writerow({'file_name' : '"""+ file_name + """',  
                         'training_time': training_time,  
                         'result_loss': result_loss,
                         'result_acc': result_acc}) 
        print('add line into architecture_results.csv')
    """)
    
    # close
    file_py.close()

# Creation CNN valide

In [38]:
create_py_file("architecture_json/architecture_valid.json")

# Creation CNN non valide

In [12]:
create_py_file("architecture_json/architecture_invalid.json")

# Create CNN json

In [68]:
directory = "architecture_json/"
for i in range(1,11):
    file_name = "archi_random_"+str(i)+"_v.json"
    print(directory+file_name)
    create_py_file(directory+file_name)
    print("")

architecture_json/archi_random_1_v.json

architecture_json/archi_random_2_v.json

architecture_json/archi_random_3_v.json

architecture_json/archi_random_4_v.json

architecture_json/archi_random_5_v.json

architecture_json/archi_random_6_v.json

architecture_json/archi_random_7_v.json

architecture_json/archi_random_8_v.json

architecture_json/archi_random_9_v.json

architecture_json/archi_random_10_v.json

